In [3]:
import pandas as pd
import numpy as np
import scipy.stats as sts
import scipy.special as ssp
import scipy.integrate as intg
import scipy.optimize as opt

ms = pd.read_csv('MacroSeries.txt',sep=',',header=None,names=['ct','kt','wt','rt'])
ct = ms['ct']
kt = ms['kt']
wt = ms['wt']
rt = ms['rt']

beta = 0.99

(a)

In [4]:
def eq3(wt, kt, alpha):

    zt = np.log(wt)-np.log(1-alpha)-alpha*np.log(kt)
    
    return zt

In [5]:
def norm_pdf(zt, rho, sigma, mu):
    zt_1 = np.append(mu, zt[:-1])

    eps = zt - rho * zt_1 - (1 - rho) * mu
    pdf_vals = np.maximum(sts.norm.pdf(eps, 0, sigma), 1e-10)
    
    return pdf_vals


In [6]:
def log_lik_norm(zt, rho, sigma, mu):
    
    pdf_vals = norm_pdf(zt, rho, sigma, mu)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

In [7]:
def crit(params, *args):
    alpha, rho, sigma, mu = params
    wt, kt = args
    zt = eq3(wt, kt, alpha)
    log_lik_val = log_lik_norm(zt, rho, sigma, mu)
    neg_log_lik_val = -log_lik_val
    
    return neg_log_lik_val

In [8]:
alpha_init = 0.3
rho_init = 0
sigma_init = 0.3
mu_init = 10

In [9]:
mle_args = (wt, kt)
params_init = np.array([alpha_init,rho_init,sigma_init,mu_init])
results = opt.minimize(crit, params_init, args=(mle_args), bounds = ((1e-10, 1-1e-10), (-1+1e-10, 1-1e-10), (1e-10, None), (1e-10, None)), method = 'L-BFGS-B')

alpha_MLE, rho_MLE, sigma_MLE, mu_MLE = results.x
inv_hess = results.hess_inv*np.identity(4) 
max_log_lik = log_lik_norm(eq3(wt, kt, alpha_MLE),rho_MLE,sigma_MLE,mu_MLE)
print('alpha_MLE=', alpha_MLE, ' rho_MLE=', rho_MLE, 'sigma_MLE=', sigma_MLE, 'mu_MLE', mu_MLE)
print('Inverse Hessian=', inv_hess)

alpha_MLE= 0.467852628313  rho_MLE= 0.712800779102 sigma_MLE= 0.0919967242399 mu_MLE 9.37837412289
Inverse Hessian= [[  1.93309199e-01   3.29806799e+00  -6.79490185e-02  -3.14215517e+00]
 [  3.29806799e+00   5.95375163e+01  -1.22518190e+00  -5.40355673e+01]
 [ -6.79490185e-02  -1.22518190e+00   2.54921923e-02   1.11345205e+00]
 [ -3.14215517e+00  -5.40355673e+01   1.11345205e+00   5.11312250e+01]]


(b)

In [10]:
def eq4(rt, kt, alpha):
    zt = np.log(rt)-np.log(alpha)+(1-alpha)*np.log(kt)
    return zt

In [11]:

def norm_pdf(zt, rho, sigma, mu):
    zt_1 = np.append(mu, zt[:-1])
    eps = zt - rho * zt_1 - (1 - rho) * mu
    pdf_vals = np.maximum(sts.norm.pdf(eps, 0, sigma), 1e-10)

    return pdf_vals


In [12]:
def log_lik_norm(zt, rho, sigma, mu):
    
    pdf_vals = norm_pdf(zt, rho, sigma, mu)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

In [13]:
def crit2(params, *args):
    alpha, rho, sigma, mu = params
    rt, kt = args
    zt = eq4(rt, kt, alpha)
    log_lik_val = log_lik_norm(zt, rho, sigma, mu)
    neg_log_lik_val = -log_lik_val
    
    return neg_log_lik_val

In [14]:
alpha_init2 = 0.3
rho_init2 = 0
sigma_init2 = 0.3
mu_init2 = 10

In [15]:
mle_args2 = (rt, kt)
params_init2 = np.array([alpha_init2, rho_init2, sigma_init2, mu_init2])
results2 = opt.minimize(crit2, params_init2, args=(mle_args2), bounds = ((1e-10, 1-1e-10), (-1+1e-10, 1-1e-10), (1e-10, None), (1e-10, None)), method = 'L-BFGS-B', options ={'ftol': 1e-10})

alpha_MLE2, rho_MLE2, sigma_MLE2, mu_MLE2 = results2.x
inv_hess2 = results2.hess_inv*np.identity(4) 
max_log_lik2 = log_lik_norm(eq4(rt, kt, alpha_MLE2), rho_MLE2, sigma_MLE2, mu_MLE2)
print('alpha_MLE=', alpha_MLE2, ' rho_MLE=', rho_MLE2, 'sigma_MLE=', sigma_MLE2, 'mu_MLE', mu_MLE2)
print('Inverse Hessian=', inv_hess2)

alpha_MLE= 0.457490101596  rho_MLE= 0.720510862973 sigma_MLE= 0.0919957014553 mu_MLE 9.37076459845
Inverse Hessian= [[  1.06453369e+02  -8.42562749e+01   2.04669283e-01  -1.91354804e+03]
 [ -8.42562749e+01   6.66929316e+01  -1.62559859e-01   1.51454508e+03]
 [  2.04669283e-01  -1.62559859e-01   2.42961250e-03  -3.67902704e+00]
 [ -1.91354804e+03   1.51454508e+03  -3.67902704e+00   3.43969028e+04]]


(c)

In [16]:
zstar = np.log(1)-np.log(alpha_MLE)+(1-alpha_MLE)*np.log(7500000)
print ('zstar =', zstar)

zstar = 9.18371490795


In [17]:
def c_pdf(x, rho_MLE, sigma_MLE, mu_MLE):

    pdf_vals    = (1/(sigma_MLE * np.sqrt(2 * np.pi)) *
                    np.exp( - (x - rho_MLE*10 - (1-rho_MLE)*mu_MLE)**2 / (2 * sigma_MLE**2)))
    
    return pdf_vals
def fit(x):
    return c_pdf(x,rho_MLE, sigma_MLE, mu_MLE)
pr_c = intg.quad(fit, zstar, np.inf)

# Alternative way:
# a = (1-alpha_MLE)*np.log(7500000)-np.log(alpha_MLE)+(rho_MLE-1)*mu_MLE-10*rho_MLE
# prc = 1-sts.norm.cdf(a, loc=0, scale= sigma_MLE)

print (pr_c)
print ('Pr(rt > 1) =', pr_c[0])

(0.999999999997931, 3.086662255654368e-11)
Pr(rt > 1) = 0.999999999997931
